# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets.

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:*

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [13]:
# your code here

import pandas as pd

df = pd.read_csv('tweets.csv', encoding='latin-1')


In [10]:
sampled_df = df.sample(n=20000, random_state=42)

In [15]:
sampled_df.columns = ['id','date','flag','user','text']


In [19]:
sampled_df.head()

,id,date,flag,user,text
4,1678756621,Sat May 02 07:38:17 PDT 2009,NO_QUERY,scotsmanrs,"One jig, five camera angles! Need I say any mo..."
4,2175496821,Mon Jun 15 01:09:08 PDT 2009,NO_QUERY,bobbyroles,@mcilwain @deathofarose We just finished Forge...
4,1826983569,Sun May 17 09:44:27 PDT 2009,NO_QUERY,Mernahuana,@420thoughts sending u good weed karma lotta ...
4,1833333992,Sun May 17 22:58:12 PDT 2009,NO_QUERY,svetin,@PRmomHawaii Doesn't it make you want to get m...
0,2263054057,Sat Jun 20 23:50:41 PDT 2009,NO_QUERY,ctaudreylim,@amirahanis I had one! But I gave it away....


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [ ]:
# your code here

import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

# Function to clean up text
def clean_up(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = ' '.join(text.split())
    return text.lower()

# Function to tokenize text
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

# Function to perform stemming and lemmatization
def stem_and_lemmatize(words):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized_words = []
    for word in words:
        stemmed_word = stemmer.stem(word)
        lemmatized_word = lemmatizer.lemmatize(word)
        stemmed_and_lemmatized_words.append((stemmed_word, lemmatized_word))
    return stemmed_and_lemmatized_words

# Function to remove stopwords
def remove_stopwords(words):
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words


df['text_processed'] = sampled_df['text'].apply(clean_up)
df['text_processed'] = df['text_processed'].apply(tokenize)
df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(remove_stopwords)

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [ ]:
# your code here

from nltk import FreqDist

# Combine all words in text_processed into a single list
all_words = [word for sublist in df['text_processed'] for word in sublist]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words from the frequency distribution
top_words = freq_dist.most_common(5000)

print(top_words)



### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

# Create a feature set and output labels
feature_set = []
output_labels = []

# Iterate over each document in the dataset
for index, row in sampled_df.iterrows():
    # Initialize feature vector for the current document
    document_features = {}
    # Iterate over each word in the top_words list
    for word, _ in top_words:
        # Check if the word is present in the current document
        # If yes, set the corresponding entry in the feature vector to 1; otherwise, set it to 0
        document_features[word] = 1 if word in row['text_processed'] else 0
    # Append the feature vector to the feature set
    feature_set.append(document_features)
    # Record the sentiment of the current document (e.g., positive or negative)
    output_labels.append(row['sentiment'])  # Assuming 'sentiment' is the name of the column containing sentiment labels

# Convert feature set and output labels into Python lists
feature_set_list = [feature_set[i] for i in range(len(feature_set))]
output_labels_list = [output_labels[i] for i in range(len(output_labels))]

# Now you have feature_set_list and output_labels_list ready to be used with nltk.NaiveBayesClassifier.train


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

import random
import nltk

# Combine feature sets and output labels
combined_data = list(zip(feature_set_list, output_labels_list))

# Shuffle the combined data to ensure randomness
random.shuffle(combined_data)

# Split the data into training and test sets (e.g., 80% training, 20% test)
train_size = int(0.8 * len(combined_data))
train_set = combined_data[:train_size]
test_set = combined_data[train_size:]

# Train the Naive Bayes classifier using the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Show the most informative features
classifier.show_most_informative_features()


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

import nltk

# Calculate the accuracy of the classifier using the test set
accuracy = nltk.classify.accuracy(classifier, test_set)

print("Classifier Accuracy:", accuracy)


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time.

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here